In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from Farm_Evaluator import getTurbLoc, loadPowerCurve, binWindResourceData, preProcessing, getAEP, checkConstraints

In [ ]:
power_curve   =  loadPowerCurve('power_curve.csv')
turb_rad = 50

In [ ]:
n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t = preProcessing(power_curve)

In [ ]:
# Years on which the data is available
year_list = ['2007','2008','2009','2013','2014','2015','2017']

wind_data_list = []
for year in year_list:
    wind_data_list.append(pd.read_csv("wind_data_{}.csv".format(year)))

In [ ]:
# Problem parameter
min_lim = 50
max_lim = 3950
turb_count = 50
penalty = 0.05

In [ ]:
def gen_coord():
    x = round(random.uniform(min_lim, max_lim),2)
    y = round(random.uniform(min_lim, max_lim),2)
    return x,y

In [ ]:
def check_violation_old(turb1, turb2):
    return np.linalg.norm(turb1 - turb2) > 8*turb_rad

In [ ]:
from shapely import affinity
from shapely.geometry.point import Point

def create_ellipse(center, lengths, angle):
    circ = Point(center).buffer(1)
    ell = affinity.scale(circ, int(lengths[0]), int(lengths[1]))
    ellr = affinity.rotate(ell, angle)
    return ellr

major = 225
minor = 200
angle = 85

def check_violation(turb1, turb2):
    '''
    Return True if intersection is empty
    '''
    ellipse1 = create_ellipse(turb1, (major,minor), angle)
    ellipse2 = create_ellipse(turb2, (major,minor), angle)
    intersect = ellipse1.intersection(ellipse2)
    return intersect.is_empty

In [ ]:
def generate_turb_locations(count):
    arr = [gen_coord()]
    for i in range(count-1):
        while True:
            new_cord = gen_coord()

            if all([check_violation(np.array(cord), np.array(new_cord)) for cord in arr]):
                arr.append(new_cord)
                break
    return arr

In [ ]:
def calculate_AEP(turb_coords):
    aep = []
    for wind_data in wind_data_list:
        wind_inst_freq =  binWindResourceData(wind_data)
        aep.append(getAEP(turb_rad, turb_coords, power_curve, wind_inst_freq, 
                  n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t))
    return sum(aep)/len(aep)

In [ ]:
class TurbGroup:
    def __init__(self):
        self.turb_coords = np.array(generate_turb_locations(turb_count))
        self.fitness = -1
        self.violation_count = 0
    
    def count_violation(self):
        for i,turb1 in enumerate(self.turb_coords):
            for turb2 in np.delete(self.turb_coords, i, axis=0):
                if  np.linalg.norm(turb1 - turb2) < 8*turb_rad:
                    self.violation_count += 1
                    
        
    def calculate_fitness(self):
        self.count_violation()
        fitness = calculate_AEP(self.turb_coords)
        fitness -= fitness*penalty*self.violation_count
        self.fitness = fitness


In [ ]:
def ga():

    pop = init_pop(POP_SIZE)
    pop = fitness(pop)
    
    for generation in range(GEN_COUNT):            
        nextgen_pop = []
        for i in range(int(POP_SIZE/2)):
            parent1 = selection(pop)
            parent2 = selection(pop)
            offspring = crossover(parent1, parent2)
            nextgen_pop.append(mutation(offspring[0]))
            nextgen_pop.append(mutation(offspring[1]))
        pop = fitness(nextgen_pop)
        best = sorted(pop, key=lambda x: x.fitness, reverse=True)
        print("Best fit in Generation no {}: {}".format(generation, best[0].fitness))
    return best[0]


In [ ]:
def init_pop(population_count):
    coord_list = []
    for i in range(population_count):
        print("Generating solution no. {}".format(i+1))
        coord_list.append(TurbGroup())
    return coord_list

In [ ]:
def fitness(population):
    for agent in population:
        agent.calculate_fitness()
    return population

In [ ]:
def selection(population):
    players = random.sample(population, TOURNAMENT_SIZE)
    sorted_players = sorted(players, key=lambda x: x.fitness, reverse=True)

    return sorted_players[0]

In [ ]:
def crossover(p1, p2):
    if random.random() < XO_PROB:
        xo_pt = random.randrange(turb_count)
        p1.turb_coords[:xo_pt],p2.turb_coords[:xo_pt] = p1.turb_coords[:xo_pt],p2.turb_coords[:xo_pt]

    return p1, p2

In [ ]:
def mutation(agent):
    if random.random() < MUT_PROB:
        return TurbGroup()
    else:
        return agent

In [ ]:
# GA parameters
POP_SIZE = 100
GEN_COUNT = 100
TOURNAMENT_SIZE = 5
MUT_PROB = 0.15
XO_PROB = 0.75

In [ ]:
# best_fit is an instance of the class TurbGroup
# you can get the location using the attribute turb_coords
# eg: best_fit.turb_coords
best_fit = ga()

Generating solution no. 1
Generating solution no. 2
Generating solution no. 3
Generating solution no. 4
Generating solution no. 5
Generating solution no. 6
Generating solution no. 7
Generating solution no. 8
Generating solution no. 9
Generating solution no. 10
Generating solution no. 11
Generating solution no. 12
Generating solution no. 13
Generating solution no. 14
Generating solution no. 15
Generating solution no. 16
Generating solution no. 17
Generating solution no. 18
Generating solution no. 19
Generating solution no. 20
Generating solution no. 21
Generating solution no. 22
Generating solution no. 23
Generating solution no. 24
Generating solution no. 25
Generating solution no. 26
Generating solution no. 27
Generating solution no. 28
Generating solution no. 29
Generating solution no. 30
Generating solution no. 31
Generating solution no. 32
Generating solution no. 33
Generating solution no. 34
Generating solution no. 35
Generating solution no. 36
Generating solution no. 37
Generating

In [ ]:
best_fit.turb_coords
plt.scatter(best_fit.turb_coords[:,0],best_fit.turb_coords[:,1], c='black')